# <i>Assignment 1  - Deeplearning 890 </i> 

Problem 1

First problem of assignment is about training a linear regression model to predic housing data submitted by [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html). In this example linear regression model is created using pytorch library. 


In [29]:
#<libraries>
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
import pandas as pd
from sklearn.preprocessing import StandardScaler 
import torch
from torch.autograd import Variable
import skimage.io as io

loading data


In [ ]:
# loading housing 
from sklearn.datasets import load_boston
data, targets = load_boston(return_X_y=True)


Process data

In [4]:

#splitting data
X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.3 , random_state=5)

# normalize data

# setting the scaler
scaler = MinMaxScaler()
scaler.fit(X_train)

# Transform train and test data according to the scaler 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Convert to floar
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)


# rearrange for one dimention
y_train = torch.tensor(y_train, dtype=torch.float).view(-1, 1).cuda()
y_test = torch.tensor(y_test, dtype=torch.float).view(-1, 1).cuda()


Model


In [5]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out


Train model

In [ ]:
##### For GPU #######

# Create the model
inputDim = 13      # takes variable 'x' 
outputDim = 1       # takes variable 'y'
model = linearRegression(inputDim, outputDim)

# Training  parameters 
learningRate = 0.001
epochs = 500

# Store information for each epoch 
loss_arr    = [] 
val_arr     = []
epoch_arr   = []


if torch.cuda.is_available():
    model.cuda()

criterion = torch.nn.MSELoss()  # mean squred error
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        X_train_input = Variable(torch.from_numpy(X_train).cuda())
        X_test_input = Variable(torch.from_numpy(X_test).cuda())
    else:
        X_train_input = Variable(torch.from_numpy(X_train))
        y_train_labels = Variable(torch.from_numpy(y_train))


    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    # Predict the output
    pred  = model(X_train_input) 
    
    # get loss for the predicted output
    loss = criterion(pred, y_train)

    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    # Validate model
    model.eval() # Put the model into evaluation mode

    with torch.no_grad():
      val_pred = model(X_test_input)
      val_loss = criterion(val_pred, y_test) # Calculate validation loss
      loss_arr.append(loss.item())
      val_arr.append(val_loss.item())
      epoch_arr.append(epoch+1)
      print(f"Epoch: {epoch} Learning rate:{learningRate} Training Loss: {loss.item()} Test Loss: {val_loss.item()}")

  

print("Training complete")

In [ ]:
plt.figure()
plt.figure(figsize=(12,8))
plt.title('Performance with a learning rate of {}'.format(learningRate))
plt.plot(epoch_arr,loss_arr,label='Train loss')

plt.plot(epoch_arr,val_arr,label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error [$medv^2$]')
plt.legend()
plt.show()


# Results

![picture](https://drive.google.com/file/d/1M-PsPmTQmOlqRIYJujZcvop0ye3IlFqP/view?usp=sharing)
Epoch: 499 Learning rate:0.001 Training Loss: 85.93895721435547 Test Loss: 94.96333312988281


![picture](https://drive.google.com/file/d/1fE_sXNBKA6I33JbAoA7HfdSC8pCsk9-N/view?usp=sharing)
Epoch: 199 Learning rate:0.01 Training Loss: 51.73222351074219 Test Loss: 65.39244079589844


![picture](https://drive.google.com/file/d/1m3gbQwjJu5WxwS1PR60XB0TYPqnMw7hn/view?usp=sharing)
Epoch: 199 Learning rate:0.1 Training Loss: 23.646242141723633 Test Loss: 36.42483139038086

Iterations needed for covergence  
* 0.001 - 300 epochs
* 0.01  - 80 epochs
* 0.1   - 30 epochs

# Discussion

According to the training information the best and lowest validation loss was given by the learning rate of 0.1. All the learning rate do converges having  higher validation losses over train loss.



In [1]:
# Please follow the links to view the images.